# Tarea corta 01

Machine Learning - Maestría de Ciencias de la computación
Profesor Dr. Jose Carranza Rojas

Estudiantes:
+ Jason Carmona (20NN...)
+ Edisson López (2013103311)

## Usando listas

In [27]:
""" Code of Jason Carmona """


' Code of Jason Carmona '

## Usando Pytorch

In [28]:
import torch



In [32]:

dtype = torch.float
device = torch.device("cpu")
# Uncomment next to run on GPU. Will be until 30x faster.
# device = torch.device("cuda:0") 


In [33]:

class LinearRegression:
    
    def __init__(self, n_features):
        ### PREV -> self.W = np.random.normal(-1,1,(n_features,1)) #weights
        self.W = torch.normal(-1,1,(n_features,1)) #weights
        ### PREV -> self.b = np.random.rand(1) #intercept
        self.b = torch.randn(1) #intercept
        
    def descent(self, delta_W, delta_b, learning_rate):
        #updating the weights with the calculated gradients
        self.W -= learning_rate*delta_W
        self.b -= learning_rate*delta_b
        
    def derivatives(self, X, Y):
        #partial derivative of cost w.r.t the weights
        delta_W = np.sum(np.multiply(((np.matmul(X,self.W)+self.b)-Y), X))*2/X.shape[0]
        #partial derivative of cost w.r.t the intercept
        delta_b = np.sum(((np.matmul(X,self.W)+self.b)-Y))*2/X.shape[0]
        return delta_W, delta_b
    
    #fit
    def train(self, X, Y, epochs=50, learning_rate=0.1):
        X = np.array(X).reshape(-1,1)
        Y = np.array(Y).reshape(-1,1)

        self.parameter_cache = []       
        for i in range(epochs):
            
            delta_W, delta_b = self.derivatives(X, Y)
            self.descent(delta_W, delta_b, learning_rate)
            
            self.parameter_cache.append(np.array((self.W,self.b)))
            
        return self.W, self.b, self.parameter_cache

    def predict(self, X):
#         print(X.shape, X.reshape(-1,1).shape)
        product = np.matmul(np.array(X).reshape(-1,1),self.W)+self.b
        return product

    




--- última línea ---